# UnifyML Quickstart

[GitHub](https://github.com/holl-/UnifyML) &nbsp; • &nbsp; [API](https://holl-.github.io/UnifyML/unifyml)

## Installation

In [ ]:
!pip install unifyml

In [1]:
from unifyml import math

## Usage without UnifyML's Tensors

In [2]:
math.sin(1.)

0.841471

In [3]:
from jax import numpy as jnp
math.sin(jnp.asarray([1.]))

DeviceArray([0.841471], dtype=float32)

In [4]:
import torch
math.sin(torch.tensor([1.]))

tensor([0.8415], device='cuda:0')

In [5]:
import tensorflow as tf
math.sin(tf.constant([1.]))

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.84147096], dtype=float32)>

In [6]:
import numpy as np
math.sin(np.asarray([1.]))

array([0.841471], dtype=float32)

## UnifyML's `Tensor`

In [7]:
...

Ellipsis

## Further Reading

*